## Part II: Stratify the data
---

#### Hierarchy:
Players will be grouped by country and then by [continent](#Map-continental-region-for-clustering)

`player` $\mapsto$ `Flag` $\mapsto$ `Region`

#### Connections: 
Player `a` is [connected](#Connect-players) to player `b` if and only if `a` and `b` have [played on a team together](#Define-connectivity-relation)

`ab` $\in$ `connections` $\iff$ `a.History` $\cup$ `b.History` $> 0$

#### Visualization:
A [variation on a radial dendogram with 2-tier hierarchical edge clusters](https://beta.observablehq.com/@youmikoh/dota-2-pro-circuit-connectivity)

$G = \{V, E\} : $ `dpc` $\mapsto V, $ `connections` $\mapsto E$ 

---

### Magically load `dpc` from `part1_collect.ipynb` in here

Cell magic `%run` will run the cells of a given notebook inside iPython as a program. This is made obvious from the output generated below, which directly mimicks the ouputs from [part1_collect.ipynb](part1_collect.ipynb). Conveniently, this notebook inherts the entire namespace of the referenced notebook including `dpc`.

In [1]:
%run part1_collect.ipynb


After collecting dpc content, 126 players of the form:

{'Alias': 'fy', 'Flag': 'cn', 'Points': 2444}

After collecting team histories, 126 players of the form:

{'Alias': 'fy',
 'Flag': 'cn',
 'History': {'LGD Gaming': [{'end': datetime.datetime(2018, 6, 10, 0, 0),
                             'start': datetime.datetime(2017, 9, 4, 0, 0)}],
             'Team VGJ': [{'end': datetime.datetime(2017, 9, 4, 0, 0),
                           'start': datetime.datetime(2016, 12, 26, 0, 0)}],
             'Vici Gaming': [{'end': datetime.datetime(2016, 3, 19, 0, 0),
                              'start': datetime.datetime(2012, 10, 21, 0, 0)},
                             {'end': datetime.datetime(2016, 12, 26, 0, 0),
                              'start': datetime.datetime(2016, 9, 16, 0, 0)}],
             'Vici Gaming Reborn': [{'end': datetime.datetime(2016, 8, 30, 0, 0),
                                     'start': datetime.datetime(2016, 3, 19, 0, 0)}]},
 'Points': 2444}


#### Map continental region for clustering

Initalize `Count` for tracking the number of connections.

In [2]:
region = {'bd':'as','be':'eu','bf':'af','bg':'eu','ba':'eu','bb':'na','wf':'oc','bl':'na','bm':'na','bn':'as','bo':'sa','bh':'as','bi':'af','bj':'af','bt':'as','jm':'na','bv':'an','bw':'af','ws':'oc','bq':'na','br':'sa','bs':'na','je':'eu','by':'eu','bz':'na','ru':'eu','rw':'af','rs':'eu','tl':'oc','re':'af','tm':'as','tj':'as','ro':'eu','tk':'oc','gw':'af','gu':'oc','gt':'na','gs':'an','gr':'eu','gq':'af','gp':'na','jp':'as','gy':'sa','gg':'eu','gf':'sa','ge':'as','gd':'na','gb':'eu','ga':'af','sv':'na','gn':'af','gm':'af','gl':'na','gi':'eu','gh':'af','om':'as','tn':'af','jo':'as','hr':'eu','ht':'na','hu':'eu','hk':'as','hn':'na','hm':'an','ve':'sa','pr':'na','ps':'as','pw':'oc','pt':'eu','sj':'eu','py':'sa','iq':'as','pa':'na','pf':'oc','pg':'oc','pe':'sa','pk':'as','ph':'as','pn':'oc','pl':'eu','pm':'na','zm':'af','eh':'af','ee':'eu','eg':'af','za':'af','ec':'sa','it':'eu','vn':'as','sb':'oc','et':'af','so':'af','zw':'af','sa':'as','es':'eu','er':'af','me':'eu','md':'eu','mg':'af','mf':'na','ma':'af','mc':'eu','uz':'as','mm':'as','ml':'af','mo':'as','mn':'as','mh':'oc','mk':'eu','mu':'af','mt':'eu','mw':'af','mv':'as','mq':'na','mp':'oc','ms':'na','mr':'af','im':'eu','ug':'af','tz':'af','my':'as','mx':'na','il':'as','fr':'eu','io':'as','sh':'af','fi':'eu','fj':'oc','fk':'sa','fm':'oc','fo':'eu','ni':'na','nl':'eu','no':'eu','na':'af','vu':'oc','nc':'oc','ne':'af','nf':'oc','ng':'af','nz':'oc','np':'as','nr':'oc','nu':'oc','ck':'oc','xk':'eu','ci':'af','ch':'eu','co':'sa','cn':'as','cm':'af','cl':'sa','cc':'as','ca':'na','cg':'af','cf':'af','cd':'af','cz':'eu','cy':'eu','cx':'as','cr':'na','cw':'na','cv':'af','cu':'na','sz':'af','sy':'as','sx':'na','kg':'as','ke':'af','ss':'af','sr':'sa','ki':'oc','kh':'as','kn':'na','km':'af','st':'af','sk':'eu','kr':'as','si':'eu','kp':'as','kw':'as','sn':'af','sm':'eu','sl':'af','sc':'af','kz':'as','ky':'na','sg':'as','se':'eu','sd':'af','do':'na','dm':'na','dj':'af','dk':'eu','vg':'na','de':'eu','ye':'as','dz':'af','us':'na','uy':'sa','yt':'af','um':'oc','lb':'as','lc':'na','la':'as','tv':'oc','tw':'as','tt':'na','tr':'as','lk':'as','li':'eu','lv':'eu','to':'oc','lt':'eu','lu':'eu','lr':'af','ls':'af','th':'as','tf':'an','tg':'af','td':'af','tc':'na','ly':'af','va':'eu','vc':'na','ae':'as','ad':'eu','ag':'na','af':'as','ai':'na','vi':'na','is':'eu','ir':'as','am':'as','al':'eu','ao':'af','aq':'an','as':'oc','ar':'sa','au':'oc','at':'eu','aw':'na','in':'as','ax':'eu','az':'as','ie':'eu','id':'as','ua':'eu','qa':'as','mz':'af'}

for player in dpc: dpc[player]['Region'] = region.get(dpc[player]['Flag'])

### Define connectivity relation

Connectivity between two players is determined essentially by a scheduling problem for each common team. By design, the team histories are already sorted in increasing order which allows the scheduling problem to perform in linear time, $O(n),\ n:$ number of combined time periods per team.

In [3]:
def teammates(duo):
    p1, p2 = duo
    h1, h2 = dpc[p1]['History'], dpc[p2]['History']
    
    common_teams = h1.keys() & h2.keys()
    while common_teams:
        team = common_teams.pop()
        th1, th2 = h1[team], h2[team]
        while th1 and th2:
            period1, period2 = th1[0], th2[0]
            if overlap(period1, period2): return True
            if period1['end'] < period2['end']: th1 = th1[1:]
            else: th2 = th2[1:]
                
    return False

def overlap(period1, period2):
    latest_start = max(period1['start'], period2['start'])
    earliest_end = min(period1['end'], period2['end'])
    delta = (earliest_end - latest_start).days + 1   
    return delta > 0

#### Connect players

Iterating over all (other) players for each player, every history pair is inspected for a connection. When a connection is found, each player increases their connection count and the duo-set is then added to `connections`. Note that directionality is irrelevant for connectivity, and thus a set (no-order) representation suffices.

To marginally optimize performance, non-connection duo-sets are added to `non-connections`. Rather than considering every duo-permutation, consider only the duo-combination by checking if the duo-set already exists in `connections` or `non-connections`; i.e connectivity for this duo has already been accounted for. This effectively halves the inspection calls at the cost of overhead checking existance in `connections` and `non-connections` as well as their storage. 

In [4]:
def connect_players_by_combinations():
    connections, non_connections = set(), set()
    for player in dpc:
        for other in dpc:
            if player!=other:
                duo = frozenset([player, other])
                if duo not in connections and duo not in non_connections:
                    if teammates(duo):
                        dpc[player]['Count'] += 1
                        dpc[other]['Count'] += 1
                        connections.add(duo)
                    else: non_connections.add(duo)
    return connections

In [5]:
def connect_players_by_permutations():
    connections = set()
    for player in dpc:
        for other in dpc:
            if player!=other:
                duo = frozenset([player, other])
                if teammates(duo):
                    dpc[player]['Count'] += 1
                    connections.add(duo)
    return connections

#### Connection performance 

In practice, connecting via combinations performs about 30% faster than permuatations as demonstrated with the `%%timeit` magics below. Even so, the process of connecting players runs in quadratic time $O(n^2),\ n:$ number of players.

In [6]:
%%timeit -n25 -r25
for player in dpc: dpc[player]['Count'] = 0
connections = connect_players_by_combinations()

20.2 ms ± 473 µs per loop (mean ± std. dev. of 25 runs, 25 loops each)


In [7]:
%%timeit -n25 -r25
for player in dpc: dpc[player]['Count'] = 0
connections = connect_players_by_permutations()

27.8 ms ± 529 µs per loop (mean ± std. dev. of 25 runs, 25 loops each)


### Mapping data for visualization

Finally, `dpc` and `connections` are mapped to the nodes and links of the graph respectively.

In [8]:
for player in dpc: dpc[player]['Count'] = 0
connections = connect_players_by_combinations()

links = connections
nodes = dpc
graph = {'nodes': nodes, 'links': links}

#### JSON encoding for objects

The graph is exported in a json file format for use with [d3.js](https://d3js.org/) to leverage its powerful visualization tools. Note that a custon json encoder should be implemented to preserve object-type data.

In [9]:
import json

class DPCEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set): 
            return list(obj)
        if isinstance(obj, frozenset): 
            return list(obj)
        elif isinstance(obj, datetime): 
            return obj.isoformat()
        else:
            return super(DPCEncoder, self).default(obj)
        
with open('data/dpc_connectivity_data.json', 'w') as outfile:  
    json.dump(graph, outfile, cls=DPCEncoder)

---
## Continue to Part III: [Visualize the data](https://beta.observablehq.com/@youmikoh/dota-2-pro-circuit-connectivity)
___
<br><br>

In [1]:
from IPython.core.display import HTML
HTML(open("css/dpc_ipynb.css", "r").read()) #IPYNB STYLING